In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train = pd.read_csv("/datasets/sberbank-russian-housing-market/train.csv")
df_macro = pd.read_csv("/datasets/sberbank-russian-housing-market/macro.csv")

In [3]:
y_train = X_train[["price_doc"]]
X_train = X_train.drop("price_doc", axis = 1)

In [4]:
X_train_sample = X_train[["full_sq", "life_sq","floor"]].fillna(0).sample(5)
y_train_sample = y_train.loc[X_train_sample.index]

## Neural Network

In [145]:
class NeuralNetwork():
    def fit(self, X, y, n_hidden, nodes, activations, lr):
        self._lr = lr
        self._X = X.values
        self._y = y.values
        self._n_hidden = n_hidden
        self._nodes = nodes
        self._weights = self._generate_weights()
        self._biases = self._generate_bias()
        self._activations = activations
        self._forward_inputs = []
        
        self._train()
        
    def _activation(self, data, activation = "relu"):
        if activation == "relu":
            def relu(data):
                return np.array([max(0,i) for i in data]).reshape(data.shape)
            return np.apply_along_axis(relu, 1, data)
        if activation == "sigmoid":
            def sigmoid(data):
                return (1/(1 + np.exp(-data))).reshape(data.shape)
            return np.apply_along_axis(sigmoid, 1, data)
    
    def _der_activation(self, points, activation = "relu"):
        if activation == "relu":
            def d_relu(point):
                return np.array([0 if y <= 0 else 1 for y in point])
            return np.apply_along_axis(d_relu, 1, points)
        if activation == "sigmoid":
            ## todo
            return
    
    def _loss_function(self, ypred, loss = "l2"):
        if loss == "mse":
            return ((ypred - self._y) ** 2).mean()
        if loss == "l2":
            return (((ypred - self._y) ** 2)/2)
    
    def _loss_jacobian(self, ypred, loss = "l2"):
        if loss == "l2":
            return (ypred - self._y)/(len(ypred))
    
    def _generate_weights(self):
        hidden_weights = []
        nodes = self._nodes
        for idx in range(1,len(nodes)):
            hidden_weights.append(0.01 * np.random.randn(nodes[idx -1], nodes[idx]))

        return hidden_weights
    
    def _generate_bias(self):
        hidden_layers = []
        nodes = self._nodes
        for i in range(self._n_hidden + 1):
            hidden_layers.append(np.zeros((nodes[i + 1], 1)))
        return hidden_layers
    
    def _forward_propagation(self):
        """
        Suppose 2 observations
        
        Suppose previous layer is 3 nodes
        Suppose current layer is 2 nodes
        
        prev shape (2,3)
        prev = ob1 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               ob2 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               
        layer shape (3,2)
        layer = [weight for current_node_1 for prev_node_1, weight for current_node_2 for prev_node_1]
                [weight for current_node_1 for prev_node_2, weight for current_node_2 for prev_node_2]
                [weight for current_node_1 for prev_node_3, weight for current_node_2 for prev_node_3]
                
        output shape (2,2) # since 2 observations and 2 layers
        output = ob1 [current_node_1 val, current_node_2 val]
                 ob2 [current_node_1 val, current_node_2 val]
                 
        Then for bias in current layer it is (2,1) since 2 nodes in current layer
        
        So for each row in output we add the bias row wise and apply the activation function to each row
        
        prev <- ouput
        
        Move onto next layer...
        """
        prev = self._X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                self._forward_inputs.append((prev, None))
                prev = (prev @ layer) + biases[idx].T,
            else:
                weight_output = (prev @ layer) + biases[idx].T
                self._forward_inputs.append((prev, weight_output))
                prev = self._activation(data = weight_output, activation = activations[idx])

        return prev
    
    def _backward_propagation(self, ypred):
        loss = self._loss_function(ypred)
        print("\nloss\n")
        print(loss)
        j = self._loss_jacobian(ypred)
        print("\nj\n")
        print(j)
                
        for i in range(len(self._forward_inputs)-1, -1, -1):
            if i != (len(self._forward_inputs) - 1):
                # activation func on all layers except the last
                der_acti = self._der_activation(self._forward_inputs[i][1])
                j = np.multiply(j,der_acti)

            x = self._forward_inputs[i][0]
            print("\nx:")
            print(x)
            jw = x.T.dot(j)
            print("\nweights before:")
            print(self._weights[i])
            self._weights[i] -= self._lr * jw
            print("\nweights after:")
            print(self._weights[i])
            # todo: update bias
            j = j.dot(self._weights[i].T)
            
        self._forward_inputs = []
        
    
    def _train(self):
        for i in range(0, 5):
            out = self._forward_propagation()
            print("\npredictions\n")
            print(out)
            self._backward_propagation(out[0])

        
        

In [147]:
INPUT_SIZE = X_train_sample.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 0.0000001
nodes = [INPUT_SIZE,3,OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]

nn = NeuralNetwork()

nn.fit(X = X_train_sample,
       y = y_train_sample,
       n_hidden = len(nodes) - 2,
       nodes = nodes,
       activations = activations,
       lr = LEARNING_RATE)


predictions

(array([[0.00841822],
       [0.01694096],
       [0.00796117],
       [0.01076127],
       [0.00590011]]),)

loss

[[4.04999999e+13]
 [2.39410962e+14]
 [3.22404499e+13]
 [6.61249999e+13]
 [2.52050000e+13]]

j

[[-1799999.99831636]
 [-4376399.99661181]
 [-1605999.99840777]
 [-2299999.99784775]
 [-1419999.99881998]]

x:
[[0.72052086 0.         0.        ]
 [1.44998765 0.         0.        ]
 [0.68140112 0.         0.        ]
 [0.83099189 0.13024594 0.        ]
 [0.50499384 0.         0.        ]]

weights before:
[[ 0.01168352]
 [ 0.00807974]
 [-0.00049087]]

weights after:
[[ 1.14822015e+00]
 [ 3.80363045e-02]
 [-4.90870544e-04]]

x:
[[ 52.  30.  11.]
 [108.  64.  19.]
 [ 48.  28.  11.]
 [ 66.  43.   6.]
 [ 34.  13.  13.]]

weights before:
[[ 0.00530963 -0.00228539 -0.01795254]
 [ 0.00894061  0.0095705  -0.01983581]
 [ 0.01601835 -0.02174161  0.02088574]]

weights after:
[[ 9.68484015e+01  5.75105707e-01 -1.79525355e-02]
 [ 5.70086097e+01  3.85749555e-01 -1.98358052e-02]